In [1]:
# %pip install ipympl

In [2]:
%matplotlib widget
import numpy as np
import caiman as cm
import matplotlib.pyplot as plt
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf
from caiman.source_extraction.cnmf.params import CNMFParams
from caiman.utils.visualization import view_quilt
import pandas as pd
import sciebo

In [3]:
sciebo.download_file_from_sciebo('https://uni-bonn.sciebo.de/s/RR7qj7tklW1rX25', 'data', 'Sue_2x_3000_40_-46.tif')
sciebo.download_file_from_sciebo('https://uni-bonn.sciebo.de/s/RR7qj7tklW1rX25', 'data', 'data_endoscope.tif')

## Data prep

In [4]:
fname = "data/Sue_2x_3000_40_-46.tif"
movie_orig = cm.load(fname)

In [5]:
# motion correction
params = CNMFParams()
motion_params = {
    'strides': (48, 48),
    'overlaps': (24, 24),
    'max_shifts': (6, 6),
    'max_deviation_rigid': 3,
    'pw_rigid': True
}
params.motion.update(motion_params)
mc = MotionCorrect(fname, **params.motion)
mc.motion_correct(save_movie=True)
mc.fname_tot_els

['data/Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap']

In [6]:
mc_fname = cm.save_memmap(
    mc.fname_tot_els,
    base_name='memmap_',
    order='C'
)

In [7]:
Yr, dims, num_frames = cm.load_memmap(mc_fname)
images = np.reshape(Yr.T, [num_frames] + list(dims), order='F')
images.shape

(3000, 170, 170)

## CNMF Model evaluation


**Signal to Noise Ratio (SNR) `min_SNR`**

What it is: This is a measure of how much the signal (in this case, the activity of neurons as indicated by calcium transients) stands out from the background noise. </br>
How it's done: For each neuron's activity trace (a graph showing their activity over time), a baseline level of noise is determined. Then, the SNR calculates how much the peaks of activity (the calcium transients) stand out compared to this baseline noise. </br>
Why it matters: Higher SNR means the neuron's activity is clearer and more distinct, making it more reliable. High SNR components are considered high quality and are less likely to be mistaken detections (false positives). </br>

**Spatial Correlation `rval_thr`**

What it is: This checks how well the shapes and locations of neurons (their "spatial footprints") detected in the data match up with where and when actual neuron activity is seen in the video (movie) of the brain. </br>
How it's done: The spatial footprints extracted are compared to the actual neuron activity in the video. This comparison generates correlation coefficients (values that measure how similar two patterns are) for the times when the neurons are active. </br>
Why it matters: High correlation means the detected spatial footprints accurately represent real neuron activity, which is crucial for valid analysis. </br>


**CNN Confidence `min_cnn_thr`**

What it is: This uses a Convolutional Neural Network (CNN), a type of artificial intelligence, to evaluate whether the detected shapes of neurons are likely to be real. </br>
How it's done: Each detected spatial component (neuron shape) is analyzed by the CNN, which has been trained on a large set of data where the correct answers (which shapes are truly neurons) are already known. </br>
Why it matters: The CNN gives a confidence score between 0 and 1 for each shape. Scores closer to 1 indicate the shape is very likely to be a real neuron, providing another layer of validation for the data. </br>

In [8]:
patch_params = {
    "rf": 15,
    "stride": 10,
    'gSig': 4,
    'K': 7    
}
params.patch.update(patch_params)
cnmf_model = cnmf.CNMF(n_processes=1, params=params)

Below steps can take a while: You can listen to this [Music](https://www.youtube.com/watch?v=HImi4zdoZrM)

In [9]:
cnmf_fit = cnmf_model.fit(images)

In [10]:
cnmf_refit = cnmf_fit.refit(images)

In [11]:
movie_corrected = cm.load(mc.mmap_file) # load motion corrected movie
correlation_image = cm.local_correlations(movie_corrected, swap_dim=False)

100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


What are the min_SNR, rval_thr, min_cnn_thr used for evaluation?

In [12]:
print("Thresholds to be used for evaluate_components()")
print(f"min_SNR = {cnmf_refit.params.quality['min_SNR']}")
print(f"rval_thr = {cnmf_refit.params.quality['rval_thr']}")
print(f"min_cnn_thr = {cnmf_refit.params.quality['min_cnn_thr']}")

Thresholds to be used for evaluate_components()
min_SNR = 2.5
rval_thr = 0.8
min_cnn_thr = 0.9


In [15]:
cnmf_refit.estimates.evaluate_components?

Signature: cnmf_refit.estimates.evaluate_components(imgs, params, dview=None)
Docstring:
Computes the quality metrics for each component and stores the
indices of the components that pass user specified thresholds. The
various thresholds and parameters can be passed as inputs. If left
empty then they are read from self.params.quality']

Args:
    imgs: np.array (possibly memory mapped, t,x,y[,z])
        Imaging data

    params: params object
        Parameters of the algorithm. The parameters in play here are
        contained in the subdictionary params.quality:

        min_SNR: float
            trace SNR threshold

        rval_thr: float
            space correlation threshold

        use_cnn: bool
            flag for using the CNN classifier

        min_cnn_thr: float
            CNN classifier threshold

Returns:
    self: estimates object
        self.idx_components: np.array
            indices of accepted components
        self.idx_components_bad: np.array
            i

In [14]:
cnmf_refit.estimates.evaluate_components(images, cnmf_refit.params);


GPU run not requested, disabling use of GPUs


FileNotFoundError: File for requested model /home/gitpod/caiman_data/model/cnn_model not found

In [ ]:
print(f"Num accepted/rejected: {len(cnmf_refit.estimates.idx_components)}, {len(cnmf_refit.estimates.idx_components_bad)}")

In [ ]:
cnmf_refit.estimates.plot_contours_nb(img=correlation_image, 
                                      idx=cnmf_refit.estimates.idx_components);

In [ ]:
cnmf_refit.estimates.nb_view_components(img=correlation_image, 
                                        idx=cnmf_refit.estimates.idx_components,
                                        cmap='gray',
                                        denoised_color='red');

In [ ]:
cnmf_refit.estimates.nb_view_components(img=correlation_image, 
                                        idx=cnmf_refit.estimates.idx_components_bad, 
                                        cmap='gray',
                                        denoised_color='red')

delta F/F

In [ ]:
cnmf_refit.estimates.detrend_df_f(quantileMin=8, 
                                    frames_window=250,
                                    flag_auto=False,
                                    use_residuals=False); 

In [ ]:
cnmf_refit.estimates.nb_view_components(img=correlation_image, 
                                        idx=cnmf_refit.estimates.idx_components,
                                        cmap='gray',
                                        denoised_color='red');

In [ ]:
frame_rate = cnmf_refit.params.data['fr']
frame_pd = 1/frame_rate
frame_times = np.linspace(0, num_frames*frame_pd, num_frames);

In [ ]:
idx_to_plot = 30
idx_accepted = cnmf_refit.estimates.idx_components
component_number = idx_accepted[idx_to_plot]
f, ax = plt.subplots(figsize=(7,2))
ax.plot(frame_times, 
        cnmf_refit.estimates.F_dff[component_number, :], 
        linewidth=0.5,
        color='k');
ax.set_xlabel('Time (s)')
ax.set_ylabel('$\Delta F/F$')
ax.set_title(f"$\Delta F/F$ for unit {component_number}");
plt.tight_layout()